# Aggregate sustainability data by OECD FUA

Sustainability data aggregated at the OECD FUA level:
- Moran et al. - 2022 - Estimating CO2 emissions for 108 000 European cities
- Huo et al. - 2022 - Carbon Monitor Cities near-real-time daily estimates of CO2 emissions from 1500 cities worldwide or Dou et al. - 2023 - Near-real-time global gridded daily CO2 emissions 2021
- Kona et al. - 2021 - Global Covenant of Mayors, a dataset of greenhouse gas emissions for 6200 cities in Europe and the Southern Mediterranean countries
- Nangini et al. - 2019 - A global dataset of CO2 emissions and ancillary data related to emissions for 343 cities


In [107]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas.tools import sjoin
from os import listdir
import os
import netCDF4 as nc
import xarray as xr
from osgeo import gdal, osr, ogr
from rasterstats import zonal_stats


In [2]:
path_folder = 'C:/Users/charl/OneDrive/Bureau/EUBUCCO/'
path_FUA_OECD = path_folder + "Data/FUA_OECD/"

In [121]:
oecd_countries = listdir(path_FUA_OECD)
FUA_OECD = gpd.read_file(path_FUA_OECD + "AUT/AUT_core_commuting.shp")
FUA_OECD = FUA_OECD.to_crs('epsg:3035')
for country in oecd_countries[1:24]:
    df2 = gpd.read_file(path_FUA_OECD + country + "/" +
                        country + "_core_commuting.shp")
    df2 = df2.to_crs('epsg:3035')
    FUA_OECD = gpd.GeoDataFrame(
        pd.concat([FUA_OECD, df2], ignore_index=True))

## Moran et al.

In [5]:
moran = pd.read_excel(
    path_folder + "Data/Moran/allcountries_onlycities_summary.xlsx")
moran2 = pd.read_excel(path_folder + "Data/Moran/allcountries_summary.xlsx")
list_countries = pd.read_excel(
    path_folder + "Data/Moran/moran_admin_level.xlsx")


,Country,Region Name,admin_level,Est. Population,Total (t CO2),CO2 per capita,airports,buildings,ets,farms,fuelstations,harbours,refineries,tiox,trains
0,austria,Abtenau,8,5751.983761,60942,10.594953,0,47936,0,0,13006,0,0,0,0
1,austria,Adlwang,8,1675.468718,12852,7.670689,0,12096,0,756,0,0,0,0,0
2,austria,Admont,8,4913.904174,70007,14.246717,0,43456,0,99,26012,0,0,0,440
3,austria,Adnet,8,3302.772923,21881,6.625039,0,21728,0,153,0,0,0,0,0
4,austria,Aflenz,8,2434.512515,24039,9.874256,0,17536,0,0,6503,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105692,ukraine,село_Червоне,8,96.933867,6776,69.903329,0,6776,0,0,0,0,0,0,0
105693,ukraine,село_Широке,8,689.810869,0,0.000000,0,0,0,0,0,0,0,0,0
105694,ukraine,село_Шляхове,8,50.298398,242,4.811286,0,242,0,0,0,0,0,0,0
105695,ukraine,село_Ясинове,8,0.000000,0,0.000000,0,0,0,0,0,0,0,0,0


In [25]:
moran_data = pd.DataFrame(columns = ['categorized', 'rname', 'population_2015', 'co2', 'ETS_activities',
       'coname', 'coco2', 'nrid', 'name', 'geometry', 'Country', 'Region Name',
       'admin_level', 'Est. Population', 'Total (t CO2)', 'CO2 per capita',
       'airports', 'buildings', 'ets', 'farms', 'fuelstations', 'harbours',
       'refineries', 'tiox', 'trains'])
for country in list_countries.moran_name:
    print(country)
    country = str(country)
    admin_level = str(
        list_countries.admin_level.loc[list_countries.moran_name == country].squeeze())
    moran3 = gpd.read_file(
        path_folder + "Data/Moran/allcountries.geojson/data/"+country+"/" + admin_level + ".geojson")
    moran3_with_data = moran3.merge(moran.loc[(moran.Country == country) & (
        moran.admin_level == int(admin_level)), :], left_on="rname", right_on="Region Name")
    moran3_with_data = moran3_with_data.drop_duplicates(subset="Region Name")
    moran3_with_data = moran3_with_data.to_crs('epsg:3035')
    moran_data = moran_data.append(moran3_with_data)

moran_with_FUA_gdf = gpd.GeoDataFrame(
    moran_data)
moran_with_FUA_gdf


austria


C:\Users\charl\AppData\Local\Temp\ipykernel_14500\4196120856.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  moran_data = moran_data.append(moran3_with_data)
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\geopandas\array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as ETRS89-extended / LAEA Europe (the single non-null crs provided).
  warnings.warn(


belgium


C:\Users\charl\AppData\Local\Temp\ipykernel_14500\4196120856.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  moran_data = moran_data.append(moran3_with_data)


bulgaria


...

C:\Users\charl\AppData\Local\Temp\ipykernel_14500\4196120856.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  moran_data = moran_data.append(moran3_with_data)


turkey


C:\Users\charl\AppData\Local\Temp\ipykernel_14500\4196120856.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  moran_data = moran_data.append(moran3_with_data)


ukraine


C:\Users\charl\AppData\Local\Temp\ipykernel_14500\4196120856.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  moran_data = moran_data.append(moran3_with_data)


In [38]:
moran_with_FUA = sjoin(FUA_OECD, moran_with_FUA_gdf, how="right")
moran_with_FUA = moran_with_FUA.loc[~np.isnan(
    moran_with_FUA.index_left), :]
moran_with_FUA.loc[:, [
    'Est. Population', 'Total (t CO2)', 'buildings', 'fuelstations']] = moran_with_FUA.loc[:, [
        'Est. Population', 'Total (t CO2)', 'buildings', 'fuelstations']].astype(float)
emissions_per_FUA = moran_with_FUA.loc[:, [
    'Est. Population', 'Total (t CO2)', 'buildings', 'fuelstations', 'fuacode']].groupby('fuacode').agg('sum')
emissions_per_FUA = emissions_per_FUA.merge(
    FUA_OECD.loc[:, ['fuacode', 'fuaname']], on="fuacode")
emissions_per_FUA.to_excel(
    path_folder+'Data/moran_aggregated_FUA_OECD.xlsx')

C:\Users\charl\AppData\Local\Temp\ipykernel_14500\1718435129.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  moran_with_FUA.loc[:, [


## Nangini et al.

In [42]:
nangini = pd.read_excel(
    path_folder + 'Data/Nangini/Scripts_and_datafiles/SCRIPTS/DATA/' + "D_FINAL.xlsx")
nangini_gdf = gpd.GeoDataFrame(nangini, geometry=gpd.points_from_xy(nangini["Longitude (others) [degrees]"], nangini["Latitude (others) [degrees]"]))
nangini_gdf = nangini_gdf.set_crs('epsg:4326')
nangini_gdf = nangini_gdf.to_crs('epsg:3035')
nangini_gdf = nangini_gdf.loc[nangini_gdf.Region == "Europe", :]
nangini_gdf


,Unnamed: 0,City name,City name (CDP),City name (carbonn),City name (PKU),City name (GEA),City name (UITP),City name (WB),Definition (CDP),Definition (carbonn),...,AQF (PKU/WB),AQF (PKU/OTHERS),PQF (CDP),PQF (carbonn),PQF (WB),PQF (WB2010),PQF (OTHERS),HQF (GEA+),HQF (OTHERS),geometry
0,0,Aarhus,Aarhus Kommune,NaN,NaN,NaN,NaN,NaN,Administrative boundary of a local government,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,POINT (4336542.333 3673720.262)
4,4,Aerøskøbing,City of Ærøskøbing,NaN,NaN,NaN,NaN,NaN,Administrative boundary of a local government,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,POINT (4342401.237 3530868.824)
11,11,Amsterdam,City of Amsterdam,NaN,NaN,Amsterdam,Amsterdam,NaN,Administrative boundary of a local government,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,POINT (3973938.099 3263000.315)
16,16,Athens,City of Athens,NaN,NaN,Athens,Athens,NaN,Administrative boundary of a local government,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,POINT (5528442.438 1764769.893)
26,26,Barcelona,NaN,Municipality of Barcelona,NaN,Barcelona,NaN,Barcelona,NaN,Municipality,...,NaN,NaN,NaN,1.0,0.0,1.0,1.0,1.0,1.0,POINT (3664996.819 2065545.393)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,309,Warsaw,City of Warsaw,NaN,NaN,NaN,NaN,NaN,Administrative boundary of a local government,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,POINT (5070354.254 3292771.254)
331,331,Zaragoza,City of Zaragoza,NaN,NaN,NaN,NaN,NaN,Administrative boundary of a local government,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,POINT (3414414.856 2126334.350)
338,338,Zürich,Stadt Zürich,NaN,NaN,Zurich,Zurich,NaN,Administrative boundary of a local government,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,1.0,POINT (4211404.309 2695894.927)
340,340,Águeda,Município de Águeda,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,POINT (2768954.137 2132192.068)


In [44]:
nangini_with_FUA = sjoin(nangini_gdf, FUA_OECD, how="right") 
nangini_with_FUA = nangini_with_FUA.loc[~np.isnan(
    nangini_with_FUA.index_left), :]

nangini_with_FUA = nangini_with_FUA.loc[:, ['City name', 'Scope-1 GHG emissions [tCO2 or tCO2-eq]',
                                            'Scope-2 (CDP) [tCO2-eq]', 'Total emissions (CDP) [tCO2-eq]',  'Population (others)',  'Population (CDP)', 'fuacode', 'fuaname']]
nangini_with_FUA


,City name,Scope-1 GHG emissions [tCO2 or tCO2-eq],Scope-2 (CDP) [tCO2-eq],Total emissions (CDP) [tCO2-eq],Population (others),Population (CDP),fuacode,fuaname
2,Graz,1218740.00,NaN,NaN,280200.0,NaN,AT002,Graz
6,Brussels,3293000.00,NaN,NaN,1175173.0,NaN,BE001,Brussels
20,Basel,783932.00,321821.00,1105753.00,197000.0,198206.0,CH003,Basel
24,Zürich,NaN,NaN,1822367.00,401144.0,404783.0,CH001,Zurich
26,Lausanne,456843.00,45730.00,499573.00,135629.0,140000.0,CH005,Lausanne
51,Magdeburg,NaN,NaN,1400000.00,235723.0,238000.0,DE019,Magdeburg
57,Hamburg,NaN,NaN,17755.00,1787408.0,1762791.0,DE002,Hamburg
105,Heidelberg,NaN,NaN,879900.00,144000.0,144948.0,DE522,Heidelberg
142,Copenhagen,NaN,NaN,1450358.00,603000.0,591481.0,DK001,Copenhagen
142,Gladsaxe Kommune,247599.00,83684.00,331283.00,66693.0,67347.0,DK001,Copenhagen


In [49]:
duplicate_FUA = set([x for x in nangini_with_FUA.fuacode if list(
        nangini_with_FUA.fuacode).count(x) > 1])
print(duplicate_FUA)
for id_dup in duplicate_FUA:
        print(id_dup)
        subset_dup = nangini_with_FUA.loc[nangini_with_FUA.fuacode == id_dup, :]
        nangini_with_FUA = nangini_with_FUA.loc[nangini_with_FUA.fuacode != id_dup, :]
        new_row = {'City name': subset_dup['City name'].iloc[0],
                   'Scope-1 GHG emissions [tCO2 or tCO2-eq]': sum(subset_dup['Scope-1 GHG emissions [tCO2 or tCO2-eq]']),
                   'Scope-2 (CDP) [tCO2-eq]': sum(subset_dup['Scope-2 (CDP) [tCO2-eq]']),
                   'Total emissions (CDP) [tCO2-eq]': sum(subset_dup['Total emissions (CDP) [tCO2-eq]']),
                   'Population (others)': sum(subset_dup['Population (others)']),
                   'Population (CDP)': sum(subset_dup['Population (CDP)']),
                   'fuacode': subset_dup['fuacode'].iloc[0],
                   'fuaname': subset_dup['fuaname'].iloc[0]}
        nangini_with_FUA = nangini_with_FUA.append(new_row, ignore_index=True)


{'DK001', 'PT001'}
DK001
PT001


C:\Users\charl\AppData\Local\Temp\ipykernel_14500\1089125330.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nangini_with_FUA = nangini_with_FUA.append(new_row, ignore_index=True)
C:\Users\charl\AppData\Local\Temp\ipykernel_14500\1089125330.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nangini_with_FUA = nangini_with_FUA.append(new_row, ignore_index=True)


In [51]:
nangini_with_FUA.to_excel(path_folder+'Data/nangini_FUA_OECD_v2.xlsx')

## Kona et al.

In [52]:
kona = pd.read_excel(path_folder + "Data/GlobalCoM.xlsx", sheet_name="Table 2")
kona_ancillary = pd.read_excel(
    path_folder + "Data/GlobalCoM.xlsx", sheet_name="Table 3")
kona = kona.merge(kona_ancillary, on="GCoM_ID")
kona_gdf = gpd.GeoDataFrame(
    kona, geometry=gpd.points_from_xy(kona["longitude"], kona["latitude"]))
kona_gdf = kona_gdf.set_crs('epsg:4326')
kona_gdf = kona_gdf.to_crs('epsg:3035')

In [53]:
kona_with_FUA = sjoin(kona_gdf, FUA_OECD, how="right")
kona_with_FUA = kona_with_FUA.loc[~np.isnan(kona_with_FUA.index_left), :]
kona_with_FUA


,index_left,GCoM_ID,emission_inventory_id,emission_inventory_sector,type_of_emissions,type_of_emission_inventory,inventory_year,population_in_the_inventory_year,activity_data_reporting_unit,activity_data,...,reference year HDD,GDP per capita at NUTS3 \n[Euro per inhabitant],GHG emissions per capita in GCoM sectors_EDGAR\n[tCO2-eq/year],reference years in EDGAR,Mitigation reduction target 2020 [%],Mitigation reduction target 2030 [%],reduction_type_id,fuacode,fuaname,geometry
1,2527.0,AT0001,1155.0,Transportation,direct_emissions,baseline_emission_inventory,1990.0,1497712.0,MWh/year,6750107.0,...,1990.0,44211.111111,4.86,1990.0,21.0,0.0,absolute,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...
1,2528.0,AT0001,1155.0,Transportation,indirect_emissions,baseline_emission_inventory,1990.0,1497712.0,MWh/year,466666.0,...,1990.0,44211.111111,4.86,1990.0,21.0,0.0,absolute,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...
1,2529.0,AT0001,1155.0,Residential buildings and facilities,direct_emissions,baseline_emission_inventory,1990.0,1497712.0,MWh/year,12104521.0,...,1990.0,44211.111111,4.86,1990.0,21.0,0.0,absolute,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...
1,2530.0,AT0001,1155.0,Residential buildings and facilities,indirect_emissions,baseline_emission_inventory,1990.0,1497712.0,MWh/year,8970791.0,...,1990.0,44211.111111,4.86,1990.0,21.0,0.0,absolute,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...
1,2531.0,AT0001,41932.0,Transportation,direct_emissions,monitoring_emission_inventory,2016.0,1853140.0,MWh/year,8972157.0,...,1990.0,44211.111111,4.86,1990.0,21.0,0.0,absolute,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,33202.0,SE0008,19862.0,Residential buildings and facilities,indirect_emissions,baseline_emission_inventory,2005.0,122062.0,MWh/year,977638.0,...,2005.0,34122.222222,3.48,2005.0,36.0,0.0,absolute,SE503,Helsingborg,MULTIPOLYGON Z (((4489459.545 3660256.193 0.00...
651,33203.0,SE0008,19862.0,Manufacturing and construction industries,direct_emissions,baseline_emission_inventory,2005.0,122062.0,MWh/year,632454.0,...,2005.0,34122.222222,3.48,2005.0,36.0,0.0,absolute,SE503,Helsingborg,MULTIPOLYGON Z (((4489459.545 3660256.193 0.00...
651,33204.0,SE0008,19862.0,Manufacturing and construction industries,indirect_emissions,baseline_emission_inventory,2005.0,122062.0,MWh/year,417571.0,...,2005.0,34122.222222,3.48,2005.0,36.0,0.0,absolute,SE503,Helsingborg,MULTIPOLYGON Z (((4489459.545 3660256.193 0.00...
651,33205.0,SE0008,19862.0,Transportation,direct_emissions,baseline_emission_inventory,2005.0,122062.0,MWh/year,1359391.0,...,2005.0,34122.222222,3.48,2005.0,36.0,0.0,absolute,SE503,Helsingborg,MULTIPOLYGON Z (((4489459.545 3660256.193 0.00...


In [54]:
kona_with_FUA = kona_with_FUA.loc[:, ['GCoM_ID', 'emission_inventory_id', 'emission_inventory_sector', 'type_of_emissions',
                                      'inventory_year', 'population_in_the_inventory_year', 'emissions', 'fuacode', 'fuaname', 'geometry']]
kona_with_FUA = kona_with_FUA.pivot(index=['GCoM_ID', 'inventory_year'], columns=[
    'emission_inventory_sector', 'type_of_emissions'], values='emissions').reset_index()


In [56]:
a = kona_with_FUA.columns
ind = pd.Index([e[0] + e[1] for e in a.tolist()])
kona_with_FUA.columns = ind
kona_with_FUA = kona_with_FUA.reset_index()
kona_with_FUA


,index,GCoM_ID,inventory_year,Transportationdirect_emissions,Transportationindirect_emissions,Residential buildings and facilitiesdirect_emissions,Residential buildings and facilitiesindirect_emissions,Municipal buildings and facilitiesdirect_emissions,Municipal buildings and facilitiesindirect_emissions,Institutional/tertiary buildings and facilitiesdirect_emissions,Institutional/tertiary buildings and facilitiesindirect_emissions,Manufacturing and construction industriesdirect_emissions,Manufacturing and construction industriesindirect_emissions,Waste/wastewaterdirect_emissions
0,0,AT0001,1990.0,1726221.181,148399.788,2.662226e+06,2020311.410,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AT0001,2016.0,2198051.098,137554.055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AT0002,2011.0,191181.340,NaN,1.327500e+05,215024.000,NaN,6232.000,87741.000,159884.000,74466.000,159856.000,NaN
3,3,AT0005,2001.0,57542.121,NaN,1.511590e+04,5845.939,883.7500,964.953,NaN,NaN,NaN,NaN,NaN
4,4,AT0010,2007.0,8031.400,0.000,1.572392e+04,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,2447,SE0041,1990.0,37383.579,16.905,3.270134e+04,2048.564,5019.7724,859.625,1590.972,939.688,NaN,NaN,NaN
2448,2448,SI0001,2008.0,695498.408,15162.654,3.551137e+05,579334.907,23586.4390,44506.363,300499.403,682113.625,252852.655,529650.808,NaN
2449,2449,SK0001,2005.0,804368.201,12475.000,3.861466e+05,552937.398,4676.3000,13003.560,258628.745,374894.928,NaN,NaN,NaN
2450,2450,SK0003,2005.0,93124.971,NaN,4.032190e+04,50758.491,2461.3150,4960.671,33203.175,17134.677,NaN,NaN,NaN


In [57]:
duplicate_kona = set(
    [x for x in kona_with_FUA.GCoM_ID if list(kona_with_FUA.GCoM_ID).count(x) > 1])

for id_dup in duplicate_kona:
    print(id_dup)
    subset_dup = kona_with_FUA.loc[kona_with_FUA.GCoM_ID == id_dup, :]
    max_year = max(subset_dup.inventory_year)
    kona_with_FUA = kona_with_FUA.loc[(kona_with_FUA.GCoM_ID != id_dup) | (
        (kona_with_FUA.GCoM_ID == id_dup) & (kona_with_FUA.inventory_year == max_year)), :]


IT3036
IT0070
IT0033
BE0072
IT1002
ES0772
GR0033
IT1480
IT0095
ES0296
ES0019
ES0936
ES0413
IT1070
ES0053
ES0312
CZ0003
ES0052
ES1166
ES0824
IT0097
NL0007
BE0156
IT0519
IT0505
ES0866
IT2597
IT0632
IT1736
IT0428
ES0337
PL0001
SE0014
IT0039
BE0285
ES0655
FI0008
IT1346
ES0120
IT0927
ES0043
IT0574
IT1216
PL0011
ES0821
IT1001
LV0003
IT2221
ES0459
IT2554
IT0196
IT1344
GR0003
IT0531
DE0005
IT0803
IT1294
IT0304
ES0079
IE0001
IT0407
IT0534
ES0304
FR0122
ES0725
IT3069
ES0141
IT0064
IT0077
IT1938
ES0802
IT4874
IT0623
ES0458
ES1809
ES0042
DE0053
ES0114
IT0049
IT1278
ES0219
PL0014
BE0081
GR0028
IT0792
DE0013
BE0173
ES1420
IT1235
CH0002
ES0210
ES0435
IT1503
ES0017
ES0565
IT0526
IT0625
ES0093
ES0827
IT0014
IT1018
ES1376
SE0011
BE0015
IT0400
ES0386
IT0878
SE0002
BE0079
ES0777
IT0653
IT0018
BE0262
BE0137
DE0004
ES0078
ES1217
ES0531
IT0970
DE0042
ES1041
PL0041
ES0647
IT0426
ES0144
ES1132
IT0038
IT0124
NL0011
IT0036
ES0968
IT1558
ES0195
DE0019
GR0016
IT1338
BE0052
FR0024
BE0189
ES0280
IT0020
ES0291
IT1329

In [58]:
kona_with_FUA = kona_with_FUA.merge(kona_gdf.loc[:, ['GCoM_ID', 'geometry', 'population_in_the_inventory_year',
                                    'inventory_year']].drop_duplicates(), on=["GCoM_ID", 'inventory_year'], how='left')
kona_with_FUA = gpd.GeoDataFrame(
    kona_with_FUA, geometry=kona_with_FUA['geometry'])

kona_with_FUA = sjoin(kona_with_FUA, FUA_OECD, how="right")
kona_with_FUA = kona_with_FUA.loc[~np.isnan(kona_with_FUA.index_left), :]
kona_with_FUA = kona_with_FUA.merge(
    kona_ancillary.loc[:, ['GCoM_ID', 'signatory name']], on="GCoM_ID")
kona_with_FUA


,index_left,index,GCoM_ID,inventory_year,Transportationdirect_emissions,Transportationindirect_emissions,Residential buildings and facilitiesdirect_emissions,Residential buildings and facilitiesindirect_emissions,Municipal buildings and facilitiesdirect_emissions,Municipal buildings and facilitiesindirect_emissions,Institutional/tertiary buildings and facilitiesdirect_emissions,Institutional/tertiary buildings and facilitiesindirect_emissions,Manufacturing and construction industriesdirect_emissions,Manufacturing and construction industriesindirect_emissions,Waste/wastewaterdirect_emissions,population_in_the_inventory_year,fuacode,fuaname,geometry,signatory name
0,0.0,1.0,AT0001,2016.0,2.198051e+06,137554.055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1853140.0,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...,Vienna
1,2.0,3.0,AT0005,2001.0,5.754212e+04,NaN,15115.9000,5845.939,883.750,964.953,NaN,NaN,NaN,NaN,NaN,13600.0,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...,Tulln
2,4.0,6.0,AT0013,2014.0,6.638119e+03,0.140,4811.7634,1493.452,204.768,189.000,2036.8356,1025.528,2262.6890,984.760,NaN,2883.0,AT001,Vienna,MULTIPOLYGON Z (((4753934.291 2875433.732 0.00...,Laxenburg
3,3.0,4.0,AT0010,2007.0,8.031400e+03,0.000,15723.9200,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5491.0,AT002,Graz,"POLYGON Z ((4734184.829 2698639.177 0.000, 473...",Gleisdorf
4,1.0,2.0,AT0002,2011.0,1.911813e+05,NaN,132750.0000,215024.000,NaN,6232.000,87741.0000,159884.000,74466.0000,159856.000,NaN,96000.0,AT006,Klagenfurt,"POLYGON Z ((4688331.015 2630159.613 0.000, 468...",Klagenfurt am Wörthersee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1737,1729.0,2433.0,SE0009,2014.0,4.589360e+05,55.350,15961.2750,47657.894,3605.835,15599.200,10368.9450,31651.699,11865.7490,15196.810,NaN,132140.0,SE004,Jonkoping,MULTIPOLYGON Z (((4540019.763 3839382.848 0.00...,Jönköping
1738,1727.0,2429.0,SE0007,1990.0,2.334780e+05,NaN,87706.4290,190640.607,8249.671,60837.831,8737.7350,95279.844,86050.5716,101986.683,NaN,122268.0,SE007,Linkoping,MULTIPOLYGON Z (((4648262.895 3958770.139 0.00...,Linköping
1739,1726.0,2428.0,SE0006,2000.0,2.287330e+05,NaN,103408.0000,522712.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140000.0,SE008,Orebro,"POLYGON Z ((4612903.257 4055628.030 0.000, 461...",Örebro
1740,1725.0,2427.0,SE0005,2015.0,7.452533e+05,11997.000,57308.6450,193312.385,32.040,40560.978,NaN,NaN,2770.0930,33133.045,NaN,145208.0,SE501,Vasteras,MULTIPOLYGON Z (((4693218.800 4139301.579 0.00...,Västerås


In [60]:
kona_with_FUA = kona_with_FUA.loc[:, ['GCoM_ID', 'inventory_year',
                                      'Municipal buildings and facilitiesindirect_emissions',
                                      'Institutional/tertiary buildings and facilitiesindirect_emissions',
                                      'Residential buildings and facilitiesindirect_emissions',
                                      'Transportationdirect_emissions',
                                      'Residential buildings and facilitiesdirect_emissions',
                                      'Municipal buildings and facilitiesdirect_emissions',
                                      'Transportationindirect_emissions', 'Waste/wastewaterdirect_emissions',
                                      'Institutional/tertiary buildings and facilitiesdirect_emissions',
                                      'Manufacturing and construction industriesdirect_emissions',
                                      'Manufacturing and construction industriesindirect_emissions',
                                      'population_in_the_inventory_year', 'fuacode',
                                      'fuaname', 'geometry',
                                      'signatory name']]

kona_with_FUA = kona_with_FUA.groupby('fuacode').agg({'Municipal buildings and facilitiesindirect_emissions': 'sum',
                                                      'Institutional/tertiary buildings and facilitiesindirect_emissions': 'sum',
                                                      'Residential buildings and facilitiesindirect_emissions': 'sum',
                                                      'Transportationdirect_emissions': 'sum',
                                                      'Residential buildings and facilitiesdirect_emissions': 'sum',
                                                      'Municipal buildings and facilitiesdirect_emissions': 'sum',
                                                      'Transportationindirect_emissions': 'sum',
                                                      'Waste/wastewaterdirect_emissions': 'sum',
                                                      'Institutional/tertiary buildings and facilitiesdirect_emissions': 'sum',
                                                      'Manufacturing and construction industriesdirect_emissions': 'sum',
                                                      'Manufacturing and construction industriesindirect_emissions': 'sum',
                                                      'population_in_the_inventory_year': 'sum',
                                                      'fuaname': 'first', 'geometry': 'first'})

kona_with_FUA.to_excel(path_folder+'Data/kona_FUA_OECD_v2.xlsx')


## Huo et al.

In [28]:
huo = pd.read_csv(path_folder + "Data/Near-real-time daily estimates of CO2 emissions from 1500 cities worldwide/carbon-monitor-cities-all-cities-FUA-v0325.csv")
huo.columns = ['city', 'country', 'date', 'sector', 'value', 'timestamp']

In [29]:
huo_2 = huo.loc[huo.date.str.startswith("2019"),:]
huo_2 = huo_2.loc[:,['city', 'country', 'sector', 'value']]
huo_2 = huo_2.groupby(['city', 'country', 'sector']
                      ).mean().unstack('sector').reset_index()


In [30]:
huo_2.columns = ['city', 'country', 'Aviation', 'transport', 'industry', 'power', 'residential']
huo_2 = huo_2.loc[:,['city', 'country', 'transport', 'residential']]


In [34]:
huo_OECD = huo_2.merge(FUA_OECD, left_on="city",
                       right_on="fuaname", how="inner")
huo_OECD.to_excel(path_folder+'Data/huo_OECD_v2.xlsx')


##  Duo et al.

In [164]:

ds_total01 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m01.nc')
ds_total02 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m02.nc')
ds_total03 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m03.nc')
ds_total04 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m04.nc')
ds_total05 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m05.nc')
ds_total06 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m06.nc')
ds_total07 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m07.nc')
ds_total08 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m08.nc')
ds_total09 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m09.nc')
ds_total10 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m10.nc')
ds_total11 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m11.nc')
ds_total12 = nc.Dataset(path_folder + 'Data/dou/CarbonMonitor_total_y2021_m12.nc')


In [165]:
total_m01 = ds_total01['emission'][:].data.mean(axis=0)
total_m02 = ds_total02['emission'][:].data.mean(axis=0)
total_m03 = ds_total03['emission'][:].data.mean(axis=0)
total_m04 = ds_total04['emission'][:].data.mean(axis=0)
total_m05 = ds_total05['emission'][:].data.mean(axis=0)
total_m06 = ds_total06['emission'][:].data.mean(axis=0)
total_m07 = ds_total07['emission'][:].data.mean(axis=0)
total_m08 = ds_total08['emission'][:].data.mean(axis=0)
total_m09 = ds_total09['emission'][:].data.mean(axis=0)
total_m10 = ds_total10['emission'][:].data.mean(axis=0)
total_m11 = ds_total11['emission'][:].data.mean(axis=0)
total_m12 = ds_total12['emission'][:].data.mean(axis=0)

latitude_here = ds_total01['latitude'][:].data
longitude_here = ds_total01['longitude'][:].data


In [166]:
total = (total_m01 + total_m02 + total_m03 + total_m04 + total_m05 + total_m06 + total_m07 + total_m08 + total_m09 + total_m10 + total_m11 + total_m12) / 12

In [167]:
grid_data = gdal.GetDriverByName('GTiff').Create(
    'grid_data', 3600,1800, 1, gdal.GDT_Int16)
grid_data.GetRasterBand(1).WriteArray(total)

# Lat/Lon WSG84 Spatial Reference System
srs = osr.SpatialReference()
srs.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')

def getGeoTransform(extent, nlines, ncols):
    resx = (extent[2] - extent[0]) / ncols
    resy = (extent[3] - extent[1]) / nlines
    return [extent[0], resx, 0, extent[3] , 0, -resy]
 

# Define the data extent (min. lon, min. lat, max. lon, max. lat)
extent = [min(longitude_here), min(latitude_here), max(longitude_here), max(latitude_here)]  # South America
print(extent)
# Setup projection and geo-transform
grid_data.SetProjection(srs.ExportToWkt())
grid_data.SetGeoTransform(getGeoTransform(extent, 1800, 3600))

[-179.95, -89.94999999998977, 179.94999999997953, 89.95]


0

In [168]:
# Save the file
file_name = 'total.tif'
print(f'Generated GeoTIFF: {file_name}')
gdal.GetDriverByName('GTiff').CreateCopy(file_name, grid_data, 0)
driver = gdal.GetDriverByName('GTiff')
driver =None


Generated GeoTIFF: total.tif


In [169]:
filename = r'total.tif'
input_raster = gdal.Open(filename)
output_raster = r'total_reprojected.tif'
warp = gdal.Warp(output_raster,input_raster,dstSRS='EPSG:3035')
warp = None # Closes the files

In [173]:
def mymean(x):
    return np.nanmean(x)

stats = gpd.GeoDataFrame(zonal_stats(
    FUA_OECD, 'total_reprojected.tif', stats=["mean"]))


c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main

In [174]:
print(stats[100:150])

             mean        mymean
100   2954.535433   2954.535433
101      0.000000           0.0
102   1633.724490    1633.72449
103   2541.400000        2541.4
104   4439.492126   4439.492126
105      0.000000           0.0
106      0.000000           0.0
107  18483.692308  18483.692308
108      0.000000           0.0
109      0.000000           0.0
110      0.000000           0.0
111      0.000000           0.0
112      0.000000           0.0
113      0.000000           0.0
114      0.000000           0.0
115   2837.204918   2837.204918
116      0.000000           0.0
117      0.000000           0.0
118   3963.000000        3963.0
119   2734.515625   2734.515625
120      0.000000           0.0
121   3447.071429   3447.071429
122   2905.631579   2905.631579
123   3985.237288   3985.237288
124      0.000000           0.0
125   5874.162242   5874.162242
126   3826.218182   3826.218182
127      0.000000           0.0
128      0.000000           0.0
129   1488.686813   1488.686813
130   25

In [146]:
huo_by_fua = FUA_OECD.loc[:,["fuacode", "fuaname"]].join(stats)
huo_by_fua.columns = ["fuacode", "fuaname", "total_emissions"]

In [147]:
ds_res01 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m01.nc')
ds_res02 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m02.nc')
ds_res03 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m03.nc')
ds_res04 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m04.nc')
ds_res05 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m05.nc')
ds_res06 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m06.nc')
ds_res07 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m07.nc')
ds_res08 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m08.nc')
ds_res09 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m09.nc')
ds_res10 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m10.nc')
ds_res11 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m11.nc')
ds_res12 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_2_Residential_y2021_m12.nc')


In [148]:
res_m01 = ds_res01['emission'][:].data.mean(axis=0)
res_m02 = ds_res02['emission'][:].data.mean(axis=0)
res_m03 = ds_res03['emission'][:].data.mean(axis=0)
res_m04 = ds_res04['emission'][:].data.mean(axis=0)
res_m05 = ds_res05['emission'][:].data.mean(axis=0)
res_m06 = ds_res06['emission'][:].data.mean(axis=0)
res_m07 = ds_res07['emission'][:].data.mean(axis=0)
res_m08 = ds_res08['emission'][:].data.mean(axis=0)
res_m09 = ds_res09['emission'][:].data.mean(axis=0)
res_m10 = ds_res10['emission'][:].data.mean(axis=0)
res_m11 = ds_res11['emission'][:].data.mean(axis=0)
res_m12 = ds_res12['emission'][:].data.mean(axis=0)

latitude_here = ds_res01['latitude'][:].data
longitude_here = ds_res01['longitude'][:].data

In [149]:
res = (res_m01 + res_m02 + res_m03 + res_m04 + res_m05 + res_m06 + res_m07 + res_m08 + res_m09 + res_m10 + res_m11 + res_m12) / 12
res.shape

(1800, 3600)

In [150]:
grid_data = gdal.GetDriverByName('GTiff').Create(
    'grid_data', 3600, 1800, 1, gdal.GDT_Int16)
grid_data.GetRasterBand(1).WriteArray(res)

# Lat/Lon WSG84 Spatial Reference System
srs = osr.SpatialReference()
srs.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')


def getGeoTransform(extent, nlines, ncols):
    resx = (extent[2] - extent[0]) / ncols
    resy = (extent[3] - extent[1]) / nlines
    return [extent[0], resx, 0, extent[3], 0, -resy]


# Define the data extent (min. lon, min. lat, max. lon, max. lat)
extent = [min(longitude_here), min(latitude_here), max(
    longitude_here), max(latitude_here)]  # South America
print(extent)
# Setup projection and geo-transform
grid_data.SetProjection(srs.ExportToWkt())
grid_data.SetGeoTransform(getGeoTransform(extent, 1800, 3600))


[-179.95, -89.94999999998977, 179.94999999997953, 89.95]


0

In [151]:
# Save the file
file_name = 'res.tif'
print(f'Generated GeoTIFF: {file_name}')
gdal.GetDriverByName('GTiff').CreateCopy(file_name, grid_data, 0)
driver = gdal.GetDriverByName('GTiff')
driver =None


Generated GeoTIFF: res.tif


In [152]:
filename = r'res.tif'
input_raster = gdal.Open(filename)
output_raster = r'res_reprojected.tif'
warp = gdal.Warp(output_raster,input_raster,dstSRS='EPSG:3035')
warp = None # Closes the files

In [153]:
stats = gpd.GeoDataFrame(zonal_stats(
    FUA_OECD, 'res_reprojected.tif', stats=["mean"]))


c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main

In [154]:
huo_by_fua = huo_by_fua.loc[:, ["fuacode",
                                "fuaname", "total_emissions"]].join(stats)
huo_by_fua.columns = ["fuacode", "fuaname", "total_emissions", "res_emissions"]


In [155]:
ds_trans01 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m01.nc')
ds_trans02 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m02.nc')
ds_trans03 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m03.nc')
ds_trans04 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m04.nc')
ds_trans05 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m05.nc')
ds_trans06 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m06.nc')
ds_trans07 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m07.nc')
ds_trans08 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m08.nc')
ds_trans09 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m09.nc')
ds_trans10 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m10.nc')
ds_trans11 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m11.nc')
ds_trans12 = nc.Dataset(
    path_folder + 'Data/dou/CarbonMonitor_3_GroundTransportation_y2021_m12.nc')


In [156]:
trans_m01 = ds_trans01['emission'][:].data.mean(axis=0)
trans_m02 = ds_trans02['emission'][:].data.mean(axis=0)
trans_m03 = ds_trans03['emission'][:].data.mean(axis=0)
trans_m04 = ds_trans04['emission'][:].data.mean(axis=0)
trans_m05 = ds_trans05['emission'][:].data.mean(axis=0)
trans_m06 = ds_trans06['emission'][:].data.mean(axis=0)
trans_m07 = ds_trans07['emission'][:].data.mean(axis=0)
trans_m08 = ds_trans08['emission'][:].data.mean(axis=0)
trans_m09 = ds_trans09['emission'][:].data.mean(axis=0)
trans_m10 = ds_trans10['emission'][:].data.mean(axis=0)
trans_m11 = ds_trans11['emission'][:].data.mean(axis=0)
trans_m12 = ds_trans12['emission'][:].data.mean(axis=0)

latitude_here = ds_trans01['latitude'][:].data
longitude_here = ds_trans01['longitude'][:].data


In [157]:
trans = (trans_m01 + trans_m02 + trans_m03 + trans_m04 + trans_m05 + trans_m06 +
       trans_m07 + trans_m08 + trans_m09 + trans_m10 + trans_m11 + trans_m12) / 12
trans.shape


(1800, 3600)

In [158]:
grid_data = gdal.GetDriverByName('GTiff').Create(
    'grid_data', 3600, 1800, 1, gdal.GDT_Int16)
grid_data.GetRasterBand(1).WriteArray(trans)

# Lat/Lon WSG84 Spatial Reference System
srs = osr.SpatialReference()
srs.ImportFromProj4('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')


def getGeoTransform(extent, nlines, ncols):
    resx = (extent[2] - extent[0]) / ncols
    resy = (extent[3] - extent[1]) / nlines
    return [extent[0], resx, 0, extent[3], 0, -resy]


# Define the data extent (min. lon, min. lat, max. lon, max. lat)
extent = [min(longitude_here), min(latitude_here), max(
    longitude_here), max(latitude_here)]  # South America
print(extent)
# Setup projection and geo-transform
grid_data.SetProjection(srs.ExportToWkt())
grid_data.SetGeoTransform(getGeoTransform(extent, 1800, 3600))

[-179.95, -89.94999999998977, 179.94999999997953, 89.95]


0

In [159]:
# Save the file
file_name = 'trans.tif'
print(f'Generated GeoTIFF: {file_name}')
gdal.GetDriverByName('GTiff').CreateCopy(file_name, grid_data, 0)
driver = gdal.GetDriverByName('GTiff')
driver = None


Generated GeoTIFF: trans.tif


In [160]:
filename = r'trans.tif'
input_raster = gdal.Open(filename)
output_raster = r'trans_reprojected.tif'
warp = gdal.Warp(output_raster, input_raster, dstSRS='EPSG:3035')
warp = None  # Closes the files


In [161]:
stats = gpd.GeoDataFrame(zonal_stats(
    FUA_OECD, 'trans_reprojected.tif', stats=["mean"]))


c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
c:\Users\charl\anaconda3\envs\geo_env\lib\site-packages\rasterstats\main

In [162]:
huo_by_fua = huo_by_fua.loc[:, ["fuacode",
                                "fuaname", "total_emissions", "res_emissions"]].join(stats)
huo_by_fua.columns = ["fuacode", "fuaname", "total_emissions", "res_emissions", "trans_emissions"]
huo_by_fua


,fuacode,fuaname,total_emissions,res_emissions,trans_emissions
0,AT004,Salzburg,3060.649485,699.536082,1127.979381
1,AT001,Vienna,3419.427379,883.255426,1306.043406
2,AT002,Graz,5047.569948,641.419689,1232.284974
3,AT003,Linz,3907.958525,438.612903,904.345622
4,AT005,Innsbruck,1693.522293,418.127389,761.407643
...,...,...,...,...,...
648,SE007,Linkoping,373.763566,16.457364,189.647287
649,SE008,Orebro,294.413333,15.951111,201.337778
650,SE501,Vasteras,691.250000,22.411111,199.722222
651,SE503,Helsingborg,932.041667,59.361111,621.347222


In [163]:
huo_by_fua.to_excel(path_folder+'Data/dou_OECD_v2.xlsx')


## Merge all databases

In [176]:
moran_OECD = pd.read_excel(
    path_folder+'Data/moran_aggregated_FUA_OECD.xlsx')
moran_OECD = moran_OECD.loc[:, ['fuacode', 'fuaname', 'Est. Population', 'Total (t CO2)',
                                'buildings', 'fuelstations']]
moran_OECD.columns = ['fuacode', 'fuaname', 'population_moran', 'scope1_moran',
                      'buildings_moran', 'transport_moran']

In [178]:
nangini_OECD = pd.read_excel(path_folder+'Data/nangini_FUA_OECD_v2.xlsx')
nangini_OECD = nangini_OECD.loc[:, ['Scope-1 GHG emissions [tCO2 or tCO2-eq]',
                                    'Total emissions (CDP) [tCO2-eq]',
                                    'Population (others)', 'Population (CDP)', 'fuacode']]
nangini_OECD.columns = ['scope1_nangini', 'total_emissions_nangini',
                        'population2_nangini', 'population1_nangini', 'fuacode']


In [180]:
kona_OECD = pd.read_excel(path_folder+'Data/kona_FUA_OECD_v2.xlsx')
kona_OECD["emissions_kona"] = np.nansum(kona_OECD.loc[:, [
    'Municipal buildings and facilitiesindirect_emissions',
    'Institutional/tertiary buildings and facilitiesindirect_emissions',
    'Residential buildings and facilitiesindirect_emissions',
    'Transportationdirect_emissions',
    'Residential buildings and facilitiesdirect_emissions',
    'Municipal buildings and facilitiesdirect_emissions',
    'Transportationindirect_emissions', 'Waste/wastewaterdirect_emissions',
    'Institutional/tertiary buildings and facilitiesdirect_emissions',
    'Manufacturing and construction industriesdirect_emissions',
    'Manufacturing and construction industriesindirect_emissions']], 1)

kona_OECD["buildings_kona"] = np.nansum(kona_OECD.loc[:, [
    'Municipal buildings and facilitiesindirect_emissions',
    'Institutional/tertiary buildings and facilitiesindirect_emissions',
    'Residential buildings and facilitiesindirect_emissions',
    'Residential buildings and facilitiesdirect_emissions',
    'Municipal buildings and facilitiesdirect_emissions',
    'Institutional/tertiary buildings and facilitiesdirect_emissions']], 1)

kona_OECD["transport_kona"] = np.nansum(kona_OECD.loc[:, [
    'Transportationdirect_emissions',
    'Transportationindirect_emissions']], 1)

kona_OECD["emissions_scope1_kona"] = np.nansum(kona_OECD.loc[:, [
    'Transportationdirect_emissions',
    'Residential buildings and facilitiesdirect_emissions',
    'Municipal buildings and facilitiesdirect_emissions',
    'Waste/wastewaterdirect_emissions',
    'Institutional/tertiary buildings and facilitiesdirect_emissions',
    'Manufacturing and construction industriesdirect_emissions']], 1)

kona_OECD["buildings_scope1_kona"] = np.nansum(kona_OECD.loc[:, [
    'Residential buildings and facilitiesdirect_emissions',
    'Municipal buildings and facilitiesdirect_emissions',
    'Institutional/tertiary buildings and facilitiesdirect_emissions']], 1)

kona_OECD["transport_scope1_kona"] = np.nansum(kona_OECD.loc[:, [
    'Transportationdirect_emissions']], 1)

kona_OECD = kona_OECD.loc[:, ['fuacode', 'population_in_the_inventory_year', 'emissions_kona', 'buildings_kona',
                              'transport_kona', 'emissions_scope1_kona', 'buildings_scope1_kona', 'transport_scope1_kona']]
kona_OECD.columns = ['fuacode', 'population_kona', 'emissions_kona', 'buildings_kona',
                     'transport_kona', 'emissions_scope1_kona', 'buildings_scope1_kona', 'transport_scope1_kona']

In [182]:
huo_OECD = pd.read_excel(path_folder+'Data/huo_OECD_v2.xlsx')
huo_OECD = huo_OECD.loc[:, ['transport', 'residential', 'fuacode']]
huo_OECD.columns = ['transport_huo', 'residential_huo', 'fuacode']


In [184]:
dou_OECD = pd.read_excel(path_folder+'Data/dou_OECD_v2.xlsx')
dou_OECD = dou_OECD.loc[:, ['fuacode', 'total_emissions',
                            'res_emissions', 'trans_emissions']]
dou_OECD.columns = ['fuacode', 'tot_emissions_dou', 'res_emissions_dou', 'trans_emissions_dou']


In [187]:
consolidated_OECD = moran_OECD.merge(huo_OECD, on="fuacode", how="outer")
consolidated_OECD = consolidated_OECD.merge(
    kona_OECD, on="fuacode", how="outer")
consolidated_OECD = consolidated_OECD.merge(
    nangini_OECD, on="fuacode", how="outer")
consolidated_OECD = consolidated_OECD.merge(
    dou_OECD, on="fuacode", how="outer")

summary = consolidated_OECD.describe()
consolidated_OECD.to_excel(path_folder+'Data/consolidated_FUA_OECD.xlsx')
